In [53]:
import os
import pandas as pd
from kernels.wrapper import MODELS, KernelModelWrapper
from dataset.ipc2023_learning_domain_info import IPC2023_LEARNING_DOMAINS, get_test_ipc2023_learning_instance_files, get_best_bound
from itertools import product
from IPython.display import display, HTML
from util.scrape_log import scrape_search_log
from util.visualise import *

In [54]:
pd.options.display.float_format = "{:.1f}".format

In [55]:
_LOG_DIR = "logs/test_ipc2023"
_DIFFICULTIES = ["easy", "medium", "hard"]
_ITERATIONS = [str(i) for i in [1]]
_MODELS = [
  # "linear-regression", 
  "linear-svr", 
  "ridge", 
  "lasso", 
  "rbf-svr", 
  "quadratic-svr", 
  "cubic-svr",
]
_REP="llg2"
CONFIGS = list(product(_MODELS, _ITERATIONS))

In [56]:
def get_data(domain):
  data = {
  }

  test_files = get_test_ipc2023_learning_instance_files()

  for config in CONFIGS:
    d = {
      "solved": [],
      "expanded": [],
      "cost": [],
      "first_h": [],
      "quality": [],
      "c_opt": [],
      "difficulty": [],
    }
    model, iterations = config

    for difficulty in _DIFFICULTIES:
      for df, pf in test_files[domain]:
        if not difficulty in pf:  # TODO generalise
          continue
        pf_name = os.path.basename(pf).replace(".pddl", "")
        log_file = f"{_LOG_DIR}/{domain}_{difficulty}_{pf_name}_-scratch-sv11-dc6693-trained_models_kernel-{model}_{_REP}_ipc2023-learning-{domain}_wl_{iterations}_0.joblib.log"
        
        if not os.path.exists(log_file):
          continue

        stats = scrape_search_log(log_file)
        for k in d:
          if k in stats:
            d[k].append(stats[k])
        
        c = stats["cost"]
        c_opt = get_best_bound(domain, difficulty, pf_name)
        quality = c_opt / c
        quality = max(quality, 0)
        d["quality"].append(quality)
        d["c_opt"].append(c_opt)
        d["difficulty"].append(difficulty)
        
    data[config] = d

  return data

In [57]:
def get_metrics_df(data):
  d = {
    "config": [],
    "quality": [],
    "solved": [],
    "tried": [],
    "e_quality": [],
    "e_solved": [],
    "e_tried": [],
    "m_quality": [],
    "m_solved": [],
    "m_tried": [],
    "h_quality": [],
    "h_solved": [],
    "h_tried": [],
  }
  for config in CONFIGS:
    d["config"].append("_".join(config))
    d["quality"].append(sum(data[config]["quality"]))
    d["solved"].append(sum(data[config]["solved"]))
    d["tried"].append(len(data[config]["solved"]))
    for difficulty in _DIFFICULTIES:
      quality = 0
      solved = 0
      tried = 0
      for i, prob_solved in enumerate(data[config]["solved"]):
        if data[config]["difficulty"][i] == difficulty:
          quality += data[config]["quality"][i]
          solved += prob_solved
          tried += 1
      d[f"{difficulty[0]}_quality"].append(quality)
      d[f"{difficulty[0]}_solved"].append(solved)
      d[f"{difficulty[0]}_tried"].append(tried)
  return pd.DataFrame(d)

In [58]:
def get_table_df():
  d = {
    "domain": [],
  }
  for config in CONFIGS:
    d['_'.join(config)] = []

  sum_scores = {config: 0 for config in CONFIGS}

  for domain in IPC2023_LEARNING_DOMAINS:
    data = get_data(domain)
    d["domain"].append(domain)
    for config in CONFIGS:
      score = sum(data[config]["quality"])
      d['_'.join(config)].append(score)
      sum_scores[config] += score
  

  d["domain"].append("sum")
  for config in CONFIGS:
    d['_'.join(config)].append(sum_scores[config])

  return pd.DataFrame(d)

In [59]:
display(get_table_df())

,domain,linear-svr_1,ridge_1,lasso_1,rbf-svr_1,quadratic-svr_1,cubic-svr_1
0,blocksworld,19.0,4.9,11.5,15.7,14.1,10.4
1,childsnack,17.1,20.7,8.8,13.0,10.0,10.0
2,ferry,63.1,36.3,63.5,35.9,63.3,57.9
3,floortile,1.9,0.9,2.9,0.0,0.0,0.0
4,miconic,28.7,28.3,30.0,28.5,28.5,28.9
5,rovers,16.4,9.7,16.4,14.5,17.4,17.7
6,satellite,5.3,29.2,7.5,8.4,44.0,42.5
7,sokoban,37.8,37.7,23.0,27.9,31.2,28.3
8,spanner,25.9,24.9,30.0,25.9,25.9,26.3
9,transport,17.4,12.2,37.0,29.5,31.6,29.1


In [60]:
for domain in IPC2023_LEARNING_DOMAINS:
  print(domain)
  data = get_data(domain)
  display(get_metrics_df(data))

blocksworld


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-svr_1,19.0,29,60,18.3,28,30,0.7,1,30,0,0,0
1,ridge_1,4.9,8,60,4.9,8,30,0.0,0,30,0,0,0
2,lasso_1,11.5,21,60,11.5,21,30,0.0,0,30,0,0,0
3,rbf-svr_1,15.7,24,60,15.7,24,30,0.0,0,30,0,0,0
4,quadratic-svr_1,14.1,23,60,14.1,23,30,0.0,0,30,0,0,0
5,cubic-svr_1,10.4,18,60,10.4,18,30,0.0,0,30,0,0,0


childsnack


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-svr_1,17.1,18,60,17.1,18,30,0,0,30,0,0,0
1,ridge_1,20.7,28,60,20.7,28,30,0,0,30,0,0,0
2,lasso_1,8.8,9,60,8.8,9,30,0,0,30,0,0,0
3,rbf-svr_1,13.0,13,60,13.0,13,30,0,0,30,0,0,0
4,quadratic-svr_1,10.0,10,60,10.0,10,30,0,0,30,0,0,0
5,cubic-svr_1,10.0,10,60,10.0,10,30,0,0,30,0,0,0


ferry


...

,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-svr_1,37.8,38,60,27.0,30,30,10.8,8,30,0,0,0
1,ridge_1,37.7,38,60,26.9,30,30,10.8,8,30,0,0,0
2,lasso_1,23.0,23,60,23.0,23,30,0.0,0,30,0,0,0
3,rbf-svr_1,27.9,30,60,27.9,30,30,0.0,0,30,0,0,0
4,quadratic-svr_1,31.2,33,60,26.7,29,30,4.5,4,30,0,0,0
5,cubic-svr_1,28.3,31,60,27.0,30,30,1.3,1,30,0,0,0


spanner


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-svr_1,25.9,30,60,25.9,30,30,0,0,30,0,0,0
1,ridge_1,24.9,29,60,24.9,29,30,0,0,30,0,0,0
2,lasso_1,30.0,30,60,30.0,30,30,0,0,30,0,0,0
3,rbf-svr_1,25.9,30,60,25.9,30,30,0,0,30,0,0,0
4,quadratic-svr_1,25.9,30,60,25.9,30,30,0,0,30,0,0,0
5,cubic-svr_1,26.3,30,60,26.3,30,30,0,0,30,0,0,0


transport


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-svr_1,17.4,26,60,15.7,23,30,1.7,3,30,0,0,0
1,ridge_1,12.2,21,60,10.5,18,30,1.6,3,30,0,0,0
2,lasso_1,37.0,33,60,33.1,30,30,3.9,3,30,0,0,0
3,rbf-svr_1,29.5,31,60,28.5,30,30,1.0,1,30,0,0,0
4,quadratic-svr_1,31.6,32,60,29.8,30,30,1.8,2,30,0,0,0
5,cubic-svr_1,29.1,30,60,29.1,30,30,0.0,0,30,0,0,0


In [61]:
os.makedirs("plots", exist_ok=True)
for domain in IPC2023_LEARNING_DOMAINS:
  try:
    data = get_data(domain)

    max_val = 0

    for config in CONFIGS:
      cs = np.array(data[config]["cost"])
      c_opts = np.array(data[config]["c_opt"])
      index = np.where(cs != -1)
      cs = cs[index]
      c_opts = c_opts[index]
      plt.scatter(c_opts, cs, label="_".join(config))
      max_val = max([max_val, max(cs), max(c_opts)])
    
    plt.title(domain)
    plt.plot([1, max_val], [1, max_val], alpha=0.5, linestyle="dotted", c='k')
    plt.ylabel("C")
    plt.xlabel("C* (not necessarily optimal)")
    plt.legend()
    # plt.show()
    plt.savefig(f"plots/{_REP}_cost_ipc2023-learning-{domain}.png")
  except:
    pass
  plt.clf()
print("c vs c* saved")

c vs c* saved


<Figure size 640x480 with 0 Axes>

In [62]:
os.makedirs("plots", exist_ok=True)
for domain in IPC2023_LEARNING_DOMAINS:
  try:
    data = get_data(domain)

    max_val = 0

    my_dict = {}

    fig, ax = plt.subplots()

    for config in CONFIGS:
      expanded = np.array(data[config]["expanded"])
      # index = np.where(expanded != -1)
      expanded = expanded[index]
      my_dict["_".join(config)] = expanded
    
    plt.yscale("log")
    plt.title(domain)
    ax.boxplot(my_dict.values())
    ax.set_xticklabels(my_dict.keys())
    # plt.ylim(bottom=1)
    # plt.show()
    plt.savefig(f"plots/{_REP}_expanded_ipc2023-learning-{domain}.png")
  except:
    pass
  plt.clf()
print("expansions saved")

expansions saved


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>